<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/query_transformations/SimpleIndexDemo-multistep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Step Query Engine

We have a multi-step query engine that's able to decompose a complex query into sequential subquestions. This
guide walks you through how to set it up!

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [26]:
%pip install llama-index-llms-openai
%pip install llama-index-agent-openai
!pip install llama-index

#### Download Data

In [27]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-14 16:29:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-03-14 16:29:41 (5.61 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



#### Load documents, build the VectorStoreIndex

In [28]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [29]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [30]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [31]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [32]:
index = VectorStoreIndex.from_documents(documents)

#### Query Index

In [33]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)

In [34]:
index_summary = "Used to answer questions about the author"

In [35]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt4)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = query_engine.query(
    "Who was in the first batch of the accelerator program the author"
    " started?",
)

> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Who is the author of the accelerator program?
> Current query: Who was in the first batch of the accelerator program the author started?
> New query: Who was in the first batch of the accelerator program started by Paul Graham?
> Current query: Who was in the first batch of the accelerator program the author started?
> New query: None


In [36]:
sub_qa = response_gpt4.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[('Who is the author of the accelerator program?', 'The author of the accelerator program is Paul Graham.'), ('Who was in the first batch of the accelerator program started by Paul Graham?', 'The first batch of the accelerator program started by Paul Graham included the founders of Reddit, Justin Kan and Emmett Shear who later founded Twitch, Aaron Swartz who had previously helped write the RSS spec, and Sam Altman who later became the second president of YC.')]


In [37]:
latest_tuple = tuples[-1]
latest_query = latest_tuple[0]
print(latest_query)

Who was in the first batch of the accelerator program started by Paul Graham?


New Technique

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

In [40]:
llm = OpenAI(temperature=0, model="gpt-4")

In [41]:
engine = index.as_query_engine(similarity_top_k=3, llm=llm)

In [42]:
from llama_index.core.tools import QueryEngineTool


query_tool_pg = QueryEngineTool.from_defaults(
    query_engine=engine,
    name="files",
    description=(
        f"Paul graham essay"
    ),
)

In [43]:
# define query plan tool
from llama_index.core.tools import QueryPlanTool
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer()
query_plan_tool = QueryPlanTool.from_defaults(
    query_engine_tools=[query_tool_pg],
    response_synthesizer=response_synthesizer,
)

In [44]:
query_plan_tool.metadata.to_openai_tool()  # to_openai_function() deprecated

{'type': 'function',
 'function': {'name': 'query_plan_tool',
  'description': '        This is a query plan tool that takes in a list of tools and executes a query plan over these tools to answer a query. The query plan is a DAG of query nodes.\n\nGiven a list of tool names and the query plan schema, you can choose to generate a query plan to answer a question.\n\nThe tool names and descriptions are as follows:\n\n\n\n        Tool Name: files\nTool Description: Paul graham essay \n        ',
  'parameters': {'type': 'object',
   'properties': {'nodes': {'title': 'Nodes',
     'description': 'The original question we are asking.',
     'type': 'array',
     'items': {'$ref': '#/definitions/QueryNode'}}},
   'required': ['nodes'],
   'definitions': {'QueryNode': {'title': 'QueryNode',
     'description': 'Query node.\n\nA query node represents a query (query_str) that must be answered.\nIt can either be answered by a tool (tool_name), or by a list of child nodes\n(child_nodes).\nThe too

In [45]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI


agent = OpenAIAgent.from_tools(
    [query_plan_tool],
    max_function_calls=10,
    llm=OpenAI(temperature=0, model="gpt-4-0613"),
    verbose=True,
)

In [46]:
response = agent.query(latest_query)

Added user message to memory: Who was in the first batch of the accelerator program started by Paul Graham?
=== Calling Function ===
Calling function: query_plan_tool with args: {
  "nodes": [
    {
      "id": 1,
      "query_str": "Who was in the first batch of the accelerator program started by Paul Graham?",
      "tool_name": "files",
      "dependencies": []
    }
  ]
}
Executing node {"id": 1, "query_str": "Who was in the first batch of the accelerator program started by Paul Graham?", "tool_name": "files", "dependencies": []}
Selected Tool: ToolMetadata(description='Paul graham essay', name='files', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>)
Executed query, got response.
Query: Who was in the first batch of the accelerator program started by Paul Graham?
Response: The first batch of the accelerator program started by Paul Graham included Reddit, Justin Kan and Emmett Shear (who later founded Twitch), Aaron Swartz (who had already helped write the RSS 

In [47]:
print(str(response))

The first batch of the accelerator program started by Paul Graham included the following individuals:

1. The team from Reddit.
2. Justin Kan and Emmett Shear, who later founded Twitch.
3. Aaron Swartz, who had already helped write the RSS spec and would later become a martyr for open access.
4. Sam Altman, who would later become the second president of Y Combinator.
